# Лабораториска вежба 5–Класификација на графови

Податочно множество –DDПодаточното множество DD се состои од1 178 протеински структури претставени како графови. Јазлите претставуваат аминокиселини. Два јазли се поврзани ако растојанието  помеѓу  соодветните  аминокиселини  е  помало  од  6ангстроми  (0.6 нанометри).Целта е да се одреди дали дадена протеинска структура е ензим или не. За вчитување на множеството искористете го следниот код:from torch_geometric.datasets import TUDatasetdata = TUDataset(root='../data/TUDataset/DD', name='DD', use_node_attr=True)

In [34]:
!pip install torch
!pip install torch_geometric

!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.3.0+cu121.html

Looking in links: https://data.pyg.org/whl/torch-2.3.0+cu121.html


In [45]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from torch_geometric.nn import to_hetero, GCNConv, Linear, SAGEConv
from torch_geometric.datasets import IMDB, Actor
from torch_geometric.loader import NeighborLoader, DataLoader
# from torch_geometric.data import DataLoader
from torch.utils.data import random_split
import torch
from torch.nn.functional import dropout
from torch_geometric.nn import to_hetero
from torch_geometric.nn import Linear, SAGEConv, global_mean_pool
from torch_geometric.datasets import TUDataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [36]:
#Од 8мата аудиториска вежба
def train(model, train_loader, val_loader, optimizer, criterion, epochs=5):
    for epoch in range(epochs):
        for i, batch in enumerate(train_loader):
            model.train()

            out = model(batch.x, batch.edge_index, batch.batch)

            loss = criterion(out, batch.y)
            loss.backward()
            train_loss = loss.item()

            optimizer.step()
            optimizer.zero_grad()

            print(f'Epoch: {epoch:03d}, Step: {i:03d}, Loss: {train_loss:.4f}')

        for i, batch in enumerate(val_loader):
            model.eval()

            out = model(batch.x, batch.edge_index, batch.batch)

            loss = criterion(out, batch.y)
            val_loss = loss.item()

            print(f'Epoch: {epoch:03d}, Step: {i:03d}, Val Loss: {val_loss:.4f}')


class GraphSAGE(torch.nn.Module):
    def __init__(self, num_classes):
        super(GraphSAGE, self).__init__()

        self.conv1 = SAGEConv((-1, -1), 64)
        self.conv2 = SAGEConv((-1, -1), 128)
        self.conv3 = SAGEConv((-1, -1), 64)

        self.linear1 = Linear(64, num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = dropout(x, p=0.5, training=self.training)
        x = self.linear1(x)

        return x



In [37]:
dataset = TUDataset(root='../data/TUDataset/DD', name='DD', use_node_attr=True)

print(dataset)

DD(1178)


## Задачи

Задача 1–Класификација на графовисо GraphSAGE (25поени) Направете модел за класификација на графовикој го имплементира GraphSAGE. Првиот дел на моделот се 2 или 3 слоеви во GraphSAGE, по што се надоврзуваат 1, 2  или  3  скриени  FC  слоеви  за  класификација  на графовите.  Направете класификација  на графовитепреку  споредба  на  различните  големини  и комбинации  на  слоеви  на  моделите.Поделете  ги  графовите  на  графови  за тренирање,  валидација  и  тестирање.Истренираниот  модел  евалуирајте  го  со метриките: точност(accuracy_score),  прецизност(precision_score),  одзив(recall_score)и F1-мерка(f1_score)

In [39]:
train_dataset, val_dataset, test_dataset = random_split(dataset, [0.8, 0.1, 0.1])


train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=32,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=32,shuffle=True)

In [40]:
model = GraphSAGE(num_classes=dataset.num_classes)
optimizer = SGD(model.parameters(), lr=0.0001)
criterion = CrossEntropyLoss()

In [41]:
train(model, train_loader, val_loader, optimizer, criterion, 15)

Epoch: 000, Step: 000, Loss: 0.7096
Epoch: 000, Step: 001, Loss: 0.7065
Epoch: 000, Step: 002, Loss: 0.6962
Epoch: 000, Step: 003, Loss: 0.6971
Epoch: 000, Step: 004, Loss: 0.7044
Epoch: 000, Step: 005, Loss: 0.7000
Epoch: 000, Step: 006, Loss: 0.6960
Epoch: 000, Step: 007, Loss: 0.7017
Epoch: 000, Step: 008, Loss: 0.7064
Epoch: 000, Step: 009, Loss: 0.6979
Epoch: 000, Step: 010, Loss: 0.6933
Epoch: 000, Step: 011, Loss: 0.6914
Epoch: 000, Step: 012, Loss: 0.6913
Epoch: 000, Step: 013, Loss: 0.6951
Epoch: 000, Step: 014, Loss: 0.7268
Epoch: 000, Step: 015, Loss: 0.7082
Epoch: 000, Step: 016, Loss: 0.7147
Epoch: 000, Step: 017, Loss: 0.6982
Epoch: 000, Step: 018, Loss: 0.6932
Epoch: 000, Step: 019, Loss: 0.6977
Epoch: 000, Step: 020, Loss: 0.6931
Epoch: 000, Step: 021, Loss: 0.6970
Epoch: 000, Step: 022, Loss: 0.6933
Epoch: 000, Step: 023, Loss: 0.7182
Epoch: 000, Step: 024, Loss: 0.6879
Epoch: 000, Step: 025, Loss: 0.6998
Epoch: 000, Step: 026, Loss: 0.6854
Epoch: 000, Step: 027, Loss:

In [83]:
# Функција за евалуација на моделот со пресметување на точност, прецизност, одзивност и F1 скор
def evaluate(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in loader:
            out = model(batch.x, batch.edge_index, batch.batch)
            preds = out.argmax(dim=1)
            all_preds.append(preds.cpu())
            all_labels.append(batch.y.cpu())

    all_preds = torch.cat(all_preds, dim=0)
    all_labels = torch.cat(all_labels, dim=0)

    # accuracy = accuracy_score(all_labels, all_preds)
    # precision = precision_score(all_labels, all_preds, average='binary')
    # recall = recall_score(all_labels, all_preds, average='binary')
    # f1 = f1_score(all_labels, all_preds, average='binary')

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='micro',zero_division=1)
    recall = recall_score(all_labels, all_preds, average='micro',zero_division=1)
    f1 = f1_score(all_labels, all_preds, average='micro',zero_division=1)

    print(f'Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}')


In [84]:
evaluate(model,test_loader)

Accuracy: 0.3076923076923077, Precision: 0.3076923076923077, Recall: 0.3076923076923077, F1 Score: 0.3076923076923077


Задача 2–Класификација на графовисо GCN(25поени) Направетеи втормодел за класификација на графовикој го имплементира GCN. Првиот дел на моделот се 2 или 3 слоеви во GCN, по што се надоврзуваат 1, 2 или 3 скриени FC слоеви за класификација на графовите. Направете класификација на графовитепреку споредба на различните големини и комбинации на слоеви на моделите.Користете ја истата поделба  како во претходната задача. Направете евалуација на истиот начин како во претходната задача.Со кој метод се добиваат подобри резултати?

In [58]:
# GCN моделот од
class GCN(torch.nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.conv1 = GCNConv(-1, 64)  # SAGEConv
        self.conv2 = GCNConv(-1, 128)  # SAGEConv

        self.linear1 = Linear(128, num_classes)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index).tanh()
        x = dropout(x, p=0.3)

        x = self.conv2(x, edge_index).tanh()
        x = dropout(x, p=0.3)

        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        x = dropout(x, p=0.5, training=self.training)
        x = self.linear1(x)

        return x

    # def forward(self, x, edge_index, batch):
    #     # 1. Obtain node embeddings
    #     x = self.conv1(x, edge_index)
    #     x = x.relu()
    #     x = self.conv2(x, edge_index)
    #     x = x.relu()
    #     x = self.conv3(x, edge_index)

    #     # 2. Readout layer
    #     x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

    #     # 3. Apply a final classifier
    #     x = dropout(x, p=0.5, training=self.training)
    #     x = self.linear1(x)

    #     return x


In [59]:
GCN_model = GCN(num_classes=dataset.num_classes)
GCN_optimizer = SGD(GCN_model.parameters(), lr=0.0001)
criterion = CrossEntropyLoss()

In [60]:
train(GCN_model, train_loader, val_loader, GCN_optimizer, criterion, 15)

Epoch: 000, Step: 000, Loss: 0.6751
Epoch: 000, Step: 001, Loss: 0.7020
Epoch: 000, Step: 002, Loss: 0.6516
Epoch: 000, Step: 003, Loss: 0.6777
Epoch: 000, Step: 004, Loss: 0.6871
Epoch: 000, Step: 005, Loss: 0.7088
Epoch: 000, Step: 006, Loss: 0.6959
Epoch: 000, Step: 007, Loss: 0.6754
Epoch: 000, Step: 008, Loss: 0.6670
Epoch: 000, Step: 009, Loss: 0.7006
Epoch: 000, Step: 010, Loss: 0.6630
Epoch: 000, Step: 011, Loss: 0.6929
Epoch: 000, Step: 012, Loss: 0.7031
Epoch: 000, Step: 013, Loss: 0.6614
Epoch: 000, Step: 014, Loss: 0.6845
Epoch: 000, Step: 015, Loss: 0.6777
Epoch: 000, Step: 016, Loss: 0.7005
Epoch: 000, Step: 017, Loss: 0.6746
Epoch: 000, Step: 018, Loss: 0.6951
Epoch: 000, Step: 019, Loss: 0.6949
Epoch: 000, Step: 020, Loss: 0.6989
Epoch: 000, Step: 021, Loss: 0.6845
Epoch: 000, Step: 022, Loss: 0.6699
Epoch: 000, Step: 023, Loss: 0.6981
Epoch: 000, Step: 024, Loss: 0.6723
Epoch: 000, Step: 025, Loss: 0.6730
Epoch: 000, Step: 026, Loss: 0.6904
Epoch: 000, Step: 027, Loss:

In [85]:
evaluate(GCN_model,test_loader)

Accuracy: 0.6923076923076923, Precision: 0.6923076923076923, Recall: 0.6923076923076923, F1 Score: 0.6923076923076923


Кроз сите метрики GCN дава подобри резултати, така да GCN е подобар.



---
GraphSAGE резултати:

Accuracy: 0.3076923076923077, Precision: 0.3076923076923077, Recall: 0.3076923076923077, F1 Score: 0.3076923076923077


---
GCN резулати:

Accuracy: 0.6923076923076923, Precision: 0.6923076923076923, Recall: 0.6923076923076923, F1 Score: 0.6923076923076923


